<a href="https://colab.research.google.com/github/yttehs123/Team-08/blob/patch-2/Back-end/Greenwood_match_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Mentee - Mentor Recommendations**

##This section informs the reader about:


1.   The criteria used for match making
2.   How each criteria is weighted in an intuitve sense
3.   How the algorithm functions

<br>


###There are two major categories for the algorithm to match make. General and technical categories of criteria. Each categorie's criteria are listed below.

**Technical Criteria**

1.   Powerpoint expertise
2.   word expertise
3.   excel expertise
4.   Industry specialisation

**General Criteria**. 

1.   Gender Identity 
2.   Ethnicity 
3.   High school/ College 
4.   if college (Industry/sector) (Select upto 3 industries) 



<br>


The user interface will ask users each general criteria as an individual qustion. Users will be asked to rate each option for each questions. 
<br> 
For e.g. On a scale of 1-3, rate who do you feel most comfortable working with. (a rating of 0 means you're not comfortable at all)

*  Male   1  2  3  0 <br>
*  Female 1  2  3  0<br>
*  Transgender 1  2  3  0<br>
*  Non Binary  1  2  3  0<br>
*  No preference Y/N<br>

note: ticking Y/N in the no preference option greys out the other options.
 <br>
 <br>
The User Interface will ask the applicants to place each of the technical criteria in order of what the applicant finds the most important to the least important in their choice on mentor/mentee.
# <br>
**The algorithm**

Each component of the algorith is explained below along with the backend code


**Fitlers**



1.   Coding Literacy
2.   Financial
3.   Stock
4.   gender
5.   ethnicity






##**Importing Libraries and the dataframe**

In [ ]:
import numpy as np
import pandas as pd

mentors = pd.read_csv("mentors_database.csv") #mentee applicant will be cross-referenced with the mentors database
mentees=pd.read_csv("mentees_database.csv") #mentees database, each record will be matched with every record of the mentors database


##Mentee attributes and Preferences

In [ ]:
def mentee_preferences_attributes(row_index, mentees):

  #Mentee preference ratings (mep)   
  #These are the preferences of the mentee 
  #These preferences define what the mentee wants from their mentor
  #mentee preferences will have the column names suffix of _preference
  powerpoint_mep_rating = mentees.iloc[row_index, mentees.columns.get_loc("powerpoint_preference")] 
  excel_mep_rating = mentees.iloc[row_index, mentees.columns.get_loc("excel_preference")] 
  word_mep_rating = mentees.iloc[row_index, mentees.columns.get_loc("word_preference")] 

  male_mep_rating = mentees.iloc[row_index, mentees.columns.get_loc("male_preference")]  
  female_mep_rating =mentees.iloc[row_index,mentees.columns.get_loc("female_preference")]  
  transgender_mep_rating =mentees.iloc[row_index, mentees.columns.get_loc("transgender_preference")]  
  nonbinary_mep_rating =mentees.iloc[row_index, mentees.columns.get_loc("nonbinary_preference")]  

  black_mep_rating =mentees.iloc[row_index, mentees.columns.get_loc("AfricanAmerican_preference")]  
  hispanic_mep_rating =mentees.iloc[row_index, mentees.columns.get_loc("hispanic_preference")]  


  #Mentee attributes (mea)  
  #These are the attributes of the mentee 
  #These attributes define who the mentee is and their skillset
  industry_mea =mentees.iloc[row_index, mentees.columns.get_loc("industry")]  
  education_mea =mentees.iloc[row_index, mentees.columns.get_loc("education")]  
  educationLevel_mea =mentees.iloc[row_index, mentees.columns.get_loc("educationLevel")]  #fresher = 1, sophomore = 2, junior = 3, senior = 4


  return powerpoint_mep_rating, excel_mep_rating, word_mep_rating, male_mep_rating, female_mep_rating, transgender_mep_rating, nonbinary_mep_rating, black_mep_rating, hispanic_mep_rating, industry_mea, education_mea, educationLevel_mea




##Mentor attributes and preferences

In [ ]:
def mentor_preferences_attributes(i, mentors):

  #Mentor attributes (ma)   
  #These are the attributes of the mentor visible 
  #These attributes define who the mentor is and their skillset
  powerpoint_ma = mentors.iloc[i, mentors .columns.get_loc("powerpoint")] 
  excel_ma = mentors.iloc[i, mentors .columns.get_loc("excel")] 
  word_ma = mentors.iloc[i, mentors .columns.get_loc("word")] 
  gender_ma = mentors.iloc[i, mentors .columns.get_loc("gender")] 
  ethnicity_ma = mentors.iloc[i, mentors .columns.get_loc("ethnicity")] 
  industry_ma = mentors.iloc[i, mentors .columns.get_loc("industry")] 

  #Mentor preferences (mp)
  #These are the preferences of the mentor defining the kind of mentee they want 
  #mentor preferences will have the column names suffix of _preference
  education_mp = mentors.iloc[i, mentors .columns.get_loc("education_preference")]  #This is their preference for high school vs college kids
  educationLevel_mp = mentors.iloc[i, mentors .columns.get_loc("educationLevel_preference")] 

  return powerpoint_ma, excel_ma, word_ma, gender_ma, ethnicity_ma, industry_ma, education_mp, educationLevel_mp

##**Dealing with the Mentee preferences**

In [ ]:
def technical_rating_criteria(ma_rating, mep_rating):
  rating_diff =  ma_rating - mep_rating
  rating=0
  if rating_diff >= 0:
      rating+=3
  elif rating_diff == -1:
    rating+= 2
  else:
    rating+= 1
  return rating


def gender_ethnicity_rating_criteria(criteria, mentorStatus, preference_rating):
  rating = 0
  if criteria == mentorStatus and preference_rating==3:
      rating+=3
  elif criteria == mentorStatus and preference_rating==2:
    rating+=2
  else:
    rating+=1

  return rating


In [ ]:
def mentee_preferences(row_index, mentees, mentors): 
  mentee_matches = pd.DataFrame(columns=['MentorID', 'Percentage Match'])


  #load current mentee preferences and attributes
  powerpoint_mep_rating, excel_mep_rating, word_mep_rating, male_mep_rating, female_mep_rating, transgender_mep_rating, nonbinary_mep_rating, black_mep_rating, hispanic_mep_rating, industry_mea, education_mea, educationLevel_mea = mentee_preferences_attributes(row_index, mentees) 
  
  #iterating through every mentor record
  for i in range(len(mentors.index)):
    #The score each criteria receives
    technical_rating=0
    general_rating=0

    #The total score in case of a perfect rating
    total_tech_score = 0
    total_general_score=0

    #load mentor record
    powerpoint_ma, excel_ma, word_ma, gender_ma, ethnicity_ma, industry_ma, education_mp, educationLevel_mp = mentor_preferences_attributes(i, mentors)

    ##   <-------------Scoring the technical criteria---------------->

    #Powerpoint, excel and word rating
    microsoft_multiplier = 1
    tech_combinations = ((powerpoint_ma, powerpoint_mep_rating), (excel_ma, excel_mep_rating), (word_ma ,word_mep_rating))
    for q in tech_combinations:
       technical_rating+=technical_rating_criteria(q[0], q[1]) * microsoft_multiplier 
       total_tech_score+=3*microsoft_multiplier 


    #Industry match rating
    #We only check for industry once
    industry_multiplier = 3 # determines the effect of the industry match on the algorithm
    industry_sets = [ set(('portfolio management', 'venture capital')), set(('asset management', 'private equity', 'financial advisor'))] #each list is a set of industries that similar to one another
    if industry_mea == industry_ma:
      technical_rating+=3*industry_multiplier
    else:
      for sets in industry_sets:
        if industry_ma in sets and industry_mea in sets:
          technical_rating+=2*industry_multiplier
        else:
          technical_rating+=1*industry_multiplier
    total_tech_score+=3*industry_multiplier

      
    ##   <-------------Scoring the general criteria---------------->

    #gender preference match rating
    #we check for every gender
    mentorGenders_menteeGenderPreference_combinations = [('male',male_mep_rating ), ('female', female_mep_rating), ('transgender', transgender_mep_rating), ('nonbinary', nonbinary_mep_rating)]

    gender_multiplier = 2
    for x in mentorGenders_menteeGenderPreference_combinations:
      general_rating+= gender_ethnicity_rating_criteria(criteria = gender_ma, mentorStatus = x[0], preference_rating = x[1]) * gender_multiplier
      total_general_score+=3 *gender_multiplier


    #ethnicity preference match rating
    #we check for every ethnicity
    mentorEthnicity_menteeEtnicityPreference_combinations = [ ('black', black_mep_rating),  ('hispanic', hispanic_mep_rating)]

    ethnicity_multiplier = 2
    for s in mentorEthnicity_menteeEtnicityPreference_combinations:
      general_rating+= gender_ethnicity_rating_criteria(criteria = ethnicity_ma, mentorStatus = s[0],  preference_rating =s[1]) * ethnicity_multiplier
      total_general_score+=3 * ethnicity_multiplier


    overall_rating = technical_rating + general_rating 
    total_rating = total_tech_score + total_general_score 

    perc_match = 100* (overall_rating/total_rating)
    #print('menteeID ' + str(row_index) + ' mentorID ' + str(i) + ' score: '+  str(perc_match))


    mentee_matches.loc[i] = [i, perc_match]

  return mentee_matches





    #TODO : create new Match dataframe, add applicant id, columns are each mentor profile id, values are percentage match
    
  


In [ ]:

#Creating a row indexer that will help make sure that the script doesn't go through the same applicants twice
row_index = 0

#Iterating through each applicant
for i in range(len(mentees.index)):
  print('menteeID '+ str(i))
  mentee_preferences(row_index, mentees, mentors)
  print(mentee_preferences(row_index, mentees, mentors))
  row_index+=1

   MentorID  Percentage Match
0       0.0         62.962963
1       1.0         53.703704
   MentorID  Percentage Match
0       0.0         61.111111
1       1.0         48.148148
